# RFからGINへの同期の失敗を解消する。

GINへの同期で失敗した場合は以下の各セッションをに順次実行してください。

コンフリクトが発生しているファイルを特定する。

In [ ]:
from utils import git_module, common, display_util

conflict_filepaths = git_module.get_conflict_filepaths()
annex_path_list = git_module.get_annex_content_file_paht_list()

# get 'git-annex' conflict path
annex_conflict_filepaths = common.get_AND_elements(conflict_filepaths, annex_path_list)
# get 'git' conflict path
git_conflict_filepaths = list(set(conflict_filepaths) - set(annex_conflict_filepaths))

display_util.display_warm('git-annex conflict : {}'.format(annex_conflict_filepaths))
display_util.display_warm('git conflict : {}'.format(git_conflict_filepaths))